In [8]:
from torchvision.transforms.functional import normalize
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# 미리 작성된 코드들은 수정할 수 없으며, 이외의 코드를 작성하시면 됩니다.

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

class Classifier(nn.Module):
    # 모델의 코드는 여기서 작성해주세요

    def __init__(self, drop_prob):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(32*32*3, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(drop_prob)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        z1 = self.linear1(x)
        a1 = self.activation(z1)
        a1 = self.dropout(a1)

        z2 = self.linear2(a1)
        a2 = self.activation(z2)
        a2 = self.dropout(a2)

        output = self.linear3(a2)

        return output


if __name__ == "__main__":
    # 학습코드는 모두 여기서 작성해주세요

    # normalization
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    
    train_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                 train=True,
                                                 transform=transform,
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                train=False,
                                                transform=transform,
                                                download=True)

    model = Classifier(0.1).to(device).train()

    #===================================
    #            train code 
    #===================================
    batch_size = 128

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    import torch.optim as optim
    optimizer = optim.SGD(params=model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()

    epochs = 90
    lmb = 0.003

    train_avg_costs = []
    
    total_batch_num = len(train_dataloader)
    
    for epoch in range(epochs):
      avg_cost = 0.0
      model.train()
      
      for b_x, b_y in train_dataloader:
        b_x = b_x.view(-1, 32*32*3).to(device)
        logits = model(b_x) # forward propagation
        loss = criterion(logits, b_y.to(device)) # get cost

        # L2 regularization
        reg = model.linear1.weight.pow(2.0).sum()
        reg += model.linear2.weight.pow(2.0).sum()
        reg += model.linear3.weight.pow(2.0).sum()
        loss += lmb * reg / len(b_x) / 2.

        optimizer.zero_grad()
        loss.backward() # backward propagation
        optimizer.step() # update parameters

        avg_cost += loss / total_batch_num
      train_avg_costs.append(avg_cost.detach().cpu())
      print('Epoch : {} / {}, cost : {}'.format(epoch+1, epochs, avg_cost))


    torch.save(model.state_dict(), 'model.pt')  # 학습된 모델을 저장하는 코드입니다.


Files already downloaded and verified
Files already downloaded and verified
Epoch : 1 / 90, cost : 2.1257476806640625
Epoch : 2 / 90, cost : 1.9149439334869385
Epoch : 3 / 90, cost : 1.837728500366211
Epoch : 4 / 90, cost : 1.7863845825195312
Epoch : 5 / 90, cost : 1.7506747245788574
Epoch : 6 / 90, cost : 1.7176330089569092
Epoch : 7 / 90, cost : 1.6888444423675537
Epoch : 8 / 90, cost : 1.6650656461715698
Epoch : 9 / 90, cost : 1.642798900604248
Epoch : 10 / 90, cost : 1.6244865655899048
Epoch : 11 / 90, cost : 1.606724739074707
Epoch : 12 / 90, cost : 1.5892096757888794
Epoch : 13 / 90, cost : 1.5739779472351074
Epoch : 14 / 90, cost : 1.5605857372283936
Epoch : 15 / 90, cost : 1.5446388721466064
Epoch : 16 / 90, cost : 1.531154751777649
Epoch : 17 / 90, cost : 1.5192245244979858
Epoch : 18 / 90, cost : 1.5044276714324951
Epoch : 19 / 90, cost : 1.494360327720642
Epoch : 20 / 90, cost : 1.4819157123565674
Epoch : 21 / 90, cost : 1.4747015237808228
Epoch : 22 / 90, cost : 1.460862755

In [9]:
# 학습된 모델의 성능을 평가하는 코드입니다.
# 아래의 코드로 평가를 진행할 예정이므로 아래의 코드가 정상 동작 해야하며, 제출전 모델의 성능을 확인하시면 됩니다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                            train=False,
                                            transform=transform,
                                            download=True)


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

classifier = Classifier(0.1).to(device)
classifier.load_state_dict(torch.load('model.pt'))
classifier.eval()


for data, label in test_dataloader:
    data = data.view(-1, 32 * 32 * 3).to(device)

    with torch.no_grad():
        logits = classifier(data)

        pred = torch.argmax(logits, dim=1)

        total = len(label)
        correct = torch.eq(pred, label.to(device)).sum()

        print("Accuracy on test set : {:.4f}%".format(100 * correct / total))

Files already downloaded and verified
Accuracy on test set : 52.5100%
